In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import copy
import lightgbm as lgb
import optuna
import statsmodels.api as sm

print("test2")

from scipy.stats import rankdata
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from optuna.integration import lightgbm as lgb_o
from sklearn.ensemble import RandomForestClassifier

import get_stock_data
pd.set_option('display.max_rows', None)

ModuleNotFoundError: No module named 'get_stock_data'

In [2]:
stock_data = pd.read_csv('stock_data.csv')

In [3]:
stock_data = stock_data.drop('Unnamed: 0', axis=1)

# データの作成

In [4]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data['year'] = stock_data['Date'].dt.year
stock_data['month'] = stock_data['Date'].dt.month
stock_data.drop(columns='Date', inplace=True)

In [5]:
id_list = [9613]
stock_data_9613 = stock_data[stock_data['Id'] == id_list[0]].reset_index(drop=True)

In [6]:
# logリターン
# 確認済み
def make_log_return(_stock_data_):
    log_return = [None]
    for i in range(1, len(_stock_data_)):
        log_return.append(np.log(_stock_data_['Close'][i] / _stock_data_['Close'][i - 1]))
    _stock_data_['log_return'] = log_return
    
make_log_return(stock_data_9613)

In [7]:
# DMI
# 確認済み
def make_dmi(_stock_data_, di_days=14, adx_days=14):
    plus_dm = [None]
    minus_dm = [None]
    tr_list = [None]
    for i in range(1, len(_stock_data_)):
        dm_plus = _stock_data_['High'][i] - _stock_data_['High'][i - 1]
        dm_minus = _stock_data_['Low'][i - 1] - _stock_data_['Low'][i]

        dm_plus = dm_plus if dm_plus > 0 else 0
        dm_minus = dm_minus if dm_minus > 0 else 0

        n = dm_plus - dm_minus
        if n > 0:
            dm_minus = 0
        elif n < 0:
            dm_plus = 0
        else:
            dm_plus = 0
            dm_minus = 0

        plus_dm.append(dm_plus)
        minus_dm.append(dm_minus)

        n_1 = abs(_stock_data_['High'][i] - _stock_data_['Low'][i])
        n_2 = abs(_stock_data_['High'][i] - _stock_data_['Close'][i - 1])
        n_3 = abs(_stock_data_['Close'][i - 1] - _stock_data_['Low'][i])

        tr = max(n_1, n_2, n_3)
        tr_list.append(tr)

    _stock_data_['tr'] = tr_list
    _stock_data_['plus_dm'] = plus_dm
    _stock_data_['minus_dm'] = minus_dm
    
    plus_di_list = [None for i in range(1, di_days)]
    minus_di_list = [None for i in range(1, di_days)]
    dx_list = [None for i in range(1, di_days)]
    for i in range(len(_stock_data_) - di_days + 1):
        plus_di = (np.sum(_stock_data_.loc[1 + i : di_days + i, 'plus_dm']) \
                    / np.sum(_stock_data_.loc[1 + i : di_days + i, 'tr'])) * 100

        minus_di = (np.sum(_stock_data_.loc[1 + i : di_days + i, 'minus_dm']) \
                    / np.sum(_stock_data_.loc[1 + i : di_days + i, 'tr'])) * 100

        dx = abs(plus_di - minus_di) / (plus_di + minus_di) * 100

        plus_di_list.append(plus_di)
        minus_di_list.append(minus_di)
        dx_list.append(dx)

    _stock_data_['plus_di'] = plus_di_list
    _stock_data_['minus_di'] = minus_di_list
    _stock_data_['dx'] = dx_list

    adx_list = [None for i in range(1, (di_days + adx_days) - 1)]
    for i in range(len(_stock_data_) - (di_days + adx_days) + 2):
        adx = np.mean(_stock_data_.loc[di_days - 1 + i : (di_days + adx_days - 2) + i , 'dx'])
        adx_list.append(adx)
    _stock_data_['adx'] = adx_list
    
    _stock_data_.drop(columns=['tr', 'plus_dm', 'minus_dm', 'dx'], inplace=True)
    
make_dmi(stock_data_9613)

In [8]:
# n日移動平均線
# 確認済み
def make_simple_moving_average(_stock_data_, average_days=[3, 5, 7, 10, 14, 18, 21]):
    for num in average_days:
        average_list = [None for i in range(1, num)]
        for i in range(len(_stock_data_) - num + 1):
            average_list.append(np.mean(_stock_data_.loc[i : num - 1 + i, 'Close']))
        _stock_data_['moving_average_' + str(num)] = average_list
    
make_simple_moving_average(stock_data_9613)

In [9]:
# 株価平均移動乖離率
# 確認済み
def make_moving_average_estrangement_rate(_stock_data_, average_days=[7, 14, 21]):
    for num in average_days:
        maer_list = []
        for i in range(len(_stock_data_)):
            maer = (_stock_data_['Close'][i] - _stock_data_['moving_average_' + str(num)][i])\
                                                / _stock_data_['moving_average_' + str(num)][i]
            maer_list.append(maer * 100)
        _stock_data_['maer_' + str(num)] = maer_list

make_moving_average_estrangement_rate(stock_data_9613)

In [10]:
# RSI
# 確認済み
def make_rsi(_stock_data_, rsi_days=[14, 21]):
    for num in rsi_days:
        db_list = [None]
        for i in range(1, len(_stock_data_)):
            db = _stock_data_['Close'][i] - _stock_data_['Close'][i - 1]
            db_list.append(db)
        _stock_data_['db'] = db_list
            
        rsi_list = [None for i in range(num)]
        rsi_overbought = [None for i in range(num)]
        rsi_oversold = [None for i in range(num)]
        for i in range(len(_stock_data_) - num):
            positive = np.sum([x for x in _stock_data_.loc[i + 1 : i + num, 'db'] if x > 0])
            negative = abs(np.sum([x for x in _stock_data_.loc[i + 1 : i + num, 'db'] if x < 0]))
            rsi = positive / (positive + negative) * 100
            rsi_list.append(rsi)
            rsi_overbought.append(1 if rsi >= 70 else 0)
            rsi_oversold.append(1 if rsi <= 30 else 0)
        _stock_data_['rsi_' + str(num)] = rsi_list
        _stock_data_['rsi_overbought_' + str(num)] = rsi_overbought
        _stock_data_['rsi_oversold_' + str(num)] = rsi_oversold
        _stock_data_.drop(columns=['db'], inplace=True)
make_rsi(stock_data_9613)

In [11]:
# RCI
# 確認済み
def make_rci(_stock_data_, rci_days = [5]):
    for num in rci_days:
        rci_list = [None for i in range(1, num)]
        rci_overbought = [None for i in range(1, num)]
        rci_oversold = [None for i in range(1, num)]
        for i in range(len(_stock_data_) - num + 1):
            rci_add = 0
            for day, rank in enumerate(rankdata(-_stock_data_.loc[i : i + num - 1, 'Close'])):
                rci_add += (day + 1 - rank) ** 2
            rci = (1 - (6 * rci_add / num / (num ** 2 - 1))) * 100
            rci_list.append(rci)
            rci_overbought.append(1 if rci >= 80 else 0)
            rci_oversold.append(1 if rci <= -80 else 0)
        _stock_data_['rci_' + str(num)] = rci_list
        _stock_data_['rci_overbought_' + str(num)] = rci_overbought
        _stock_data_['rci_oversold_' + str(num)] = rci_oversold

make_rci(stock_data_9613)

In [12]:
# 強弱レシオ
# 確認済み

def make_intensity_ratio(_stock_data_, intensity_days=[21]):
    for num in intensity_days:
        _stock_data_['high_open'] = _stock_data_['High'] - _stock_data_['Open']
        _stock_data_['open_low'] = _stock_data_['Open'] - _stock_data_['Low']

        a_ratio = [None for i in range(1, num)]
        for i in range(len(_stock_data_) - num + 1):
            high_open_sum = np.sum(_stock_data_.loc[i : num + i - 1, 'high_open'])
            open_low_sum = np.sum(_stock_data_.loc[i : num + i - 1, 'open_low'])
            a_ratio.append(high_open_sum / open_low_sum * 100)
        _stock_data_['a_ratio'] = a_ratio

        high_close = [None]
        close_low = [None]
        high_middle = [None]
        middle_low = [None]
        for i in range(1, len(_stock_data_)):
            high_close.append(_stock_data_['High'][i] - _stock_data_['Close'][i - 1])
            close_low.append(_stock_data_['Close'][i - 1] - _stock_data_['Low'][i])
            high_middle.append(_stock_data_['High'][i] \
                               - (_stock_data_['High'][i - 1] + _stock_data_['Low'][i - 1]) / 2 \
                               - _stock_data_['Low'][i - 1])
            middle_low.append((_stock_data_['High'][i - 1] + _stock_data_['Low'][i - 1]) / 2 \
                              + _stock_data_['Low'][i - 1]
                              - _stock_data_['Low'][i])

        _stock_data_['high_close'] = high_close
        _stock_data_['close_low'] = close_low
        _stock_data_['high_middle'] = high_middle
        _stock_data_['middle_low'] = middle_low

        b_ratio = [None for i in range(num)]
        c_ratio = [None for i in range(num)]
        for i in range(1, len(_stock_data_) - num + 1):
            high_close_sum = np.sum(_stock_data_.loc[i : num + i - 1, 'high_close'])
            close_low_sum = np.sum(_stock_data_.loc[i : num + i - 1, 'close_low'])
            high_middle_sum = np.sum(_stock_data_.loc[i : num + i - 1, 'high_middle'])
            middle_low_sum = np.sum(_stock_data_.loc[i : num + i - 1, 'middle_low'])

            b_ratio.append(high_close_sum / close_low_sum * 100)
            c_ratio.append(high_middle_sum / middle_low_sum * 100)
        _stock_data_['b_ratio'] = b_ratio
        _stock_data_['c_ratio'] = c_ratio
        
        _stock_data_.drop(columns=['high_open', 'open_low', 'high_close',
                                   'close_low', 'high_middle', 'middle_low'], inplace=True)
make_intensity_ratio(stock_data_9613)

In [13]:
# ストキャスティクス
# 確認済み
def make_stochastic_oscillator(_stock_data_, fast_days=14, m=7, l=7):
    fast_K_list = [None for i in range(1, fast_days)]
    close_low_list = [None for i in range(1, fast_days)]
    high_low_list = [None for i in range(1, fast_days)]
    slow_K_list = [None for i in range(2, fast_days + m)]
    slow_D_list = [None for i in range(3, fast_days + m + l)]
    for i in range(len(_stock_data_) - fast_days + 1):
        recent_close = _stock_data_['Close'][fast_days + i - 1]
        low_fast = min(_stock_data_.loc[i : fast_days + i - 1, 'Low'])
        high_fast = max(_stock_data_.loc[i : fast_days + i - 1, 'High'])
        
        close_low = recent_close - low_fast
        high_low = high_fast - low_fast
        fast_K = close_low / high_low * 100
        
        fast_K_list.append(fast_K)
        close_low_list.append(close_low)
        high_low_list.append(high_low)
        
    _stock_data_['fast_K'] = fast_K_list
    _stock_data_['close_low'] = close_low_list
    _stock_data_['high_low'] = high_low_list
    
    for i in range(len(_stock_data_) - fast_days - m + 2):
        slow_K = np.sum(_stock_data_.loc[fast_days + i - 1 : fast_days + m + i - 2, 'close_low'])\
                / np.sum(_stock_data_.loc[fast_days + i - 1 : fast_days + m + i - 2, 'high_low']) * 100
        slow_K_list.append(slow_K)
    _stock_data_['slow_K'] = slow_K_list
        
    for i in range(len(_stock_data_) - fast_days - m - l + 3):
        slow_D_list.append(np.mean(_stock_data_.loc[fast_days + m + i - 2 : fast_days + m + l + i - 3, 'slow_K']))
    _stock_data_['slow_D'] = slow_D_list
    
    _stock_data_.drop(columns=['close_low', 'high_low'], inplace=True)
make_stochastic_oscillator(stock_data_9613)

In [14]:
# OLSによるトレンドフラグの作成
# 確認済み
def make_target_col_ols(_stock_data_, roc=0.05, date_range=21):
    trend_flag_ols = []
    for i in range(len(_stock_data_) - date_range + 1):
        x = sm.add_constant([n for n in range(date_range)])
        y = _stock_data_.loc[i : date_range + i - 1, 'Close']

        result = sm.OLS(y, x).fit()

        if result.params[1] >= roc * _stock_data_['Close'][i] / date_range and result.pvalues[1] <= 0.01 \
        and not (_stock_data_['Close'][i] - result.params[0] < 0 and result.pvalues[0] <= 0.01):
            trend_flag_ols.append(1)
        elif result.params[1] <= -roc * _stock_data_['Close'][i] / date_range and result.pvalues[1] <= 0.01 \
        and not (_stock_data_['Close'][i] - result.params[0] > 0 and result.pvalues[0] <= 0.01):
            trend_flag_ols.append(2)
        else:
            trend_flag_ols.append(0)

    [trend_flag_ols.append(None) for i in range(1, date_range)]
    _stock_data_['trend_flag_ols'] = trend_flag_ols
make_target_col_ols(stock_data_9613)

In [32]:
####### test_data #######
def make_dataset(_stock_data_):
    target_col = 'trend_flag_ols'
    exclude_cols = ['Id', 'Open', 'High' , 'Low', 'Adj Close', 'trend_flag_ols']
    feature_cols = []
    for col in _stock_data_.columns:
        if col not in exclude_cols:
            feature_cols.append(col)

    x_train_val = _stock_data_[(_stock_data_['year'] >= 2011) & (_stock_data_['year'] <= 2018)][feature_cols]
    y_train_val = _stock_data_[(_stock_data_['year'] >= 2011) & (_stock_data_['year'] <= 2018)][target_col]
    
    x_test = feature_data = _stock_data_[_stock_data_['year'] == 2019][feature_cols]
    y_test = feature_data = _stock_data_[_stock_data_['year'] == 2019][target_col]
    
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val, test_size=0.3, random_state=1234)
    
    return x_train_val, y_train_val, x_train, y_train, x_val, y_val, x_test, y_test
    
x_train_val, y_train_val, x_train, y_train, x_val, y_val, x_test, y_test = make_dataset(stock_data_9613)

# モデルの作成

## Light GBM

In [55]:
def metric_precision(preds, data):
    y_true = data.get_label()
    num_labels = 3
    y_pred = np.argmax(preds.reshape(num_labels, len(preds) // num_labels), axis=0)
    matrix = confusion_matrix(y_pred, y_true)
    if (np.sum(matrix[1]) + np.sum(matrix[2])) == 0:
        precision = 0
    else:
        precision = (matrix[1][1] + matrix[2][2]) / (np.sum(matrix[1]) + np.sum(matrix[2]))
    return 'my_precision', precision, True

def objective(trial):
    dtrain = lgb.Dataset(x_train, label=y_train)
    eval_data = lgb.Dataset(x_val, label=y_val)

    param = {
        "objective": "multiclass",
        "num_class": 3,
        "metric": 'None',
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 1234,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100)
    }

    lgb_optuna_model = lgb.train(param, 
                                 dtrain, 
                                 valid_names=['valid'], 
                                 valid_sets=[eval_data],
                                 feval=metric_precision,
                                 num_boost_round=10000,
                                 early_stopping_rounds=200,
                                 )

    
    y_pred = np.argmax(lgb_optuna_model.predict(x_val), axis=1)
    matrix = confusion_matrix(y_pred, y_val)
    if (np.sum(matrix[1]) + np.sum(matrix[2])) == 0:
        precision = 0
    else:
        precision = (matrix[1][1] + matrix[2][2]) / (np.sum(matrix[1]) + np.sum(matrix[2]))
    return precision
    
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

[1]	valid's my_precision: 0
Training until validation scores don't improve for 200 rounds
[2]	valid's my_precision: 0
[3]	valid's my_precision: 0
[4]	valid's my_precision: 0
[5]	valid's my_precision: 0
[6]	valid's my_precision: 0
[7]	valid's my_precision: 0
[8]	valid's my_precision: 0
[9]	valid's my_precision: 0
[10]	valid's my_precision: 0
[11]	valid's my_precision: 0
[12]	valid's my_precision: 0
[13]	valid's my_precision: 0
[14]	valid's my_precision: 0
[15]	valid's my_precision: 0
[16]	valid's my_precision: 0
[17]	valid's my_precision: 1
[18]	valid's my_precision: 0
[19]	valid's my_precision: 0
[20]	valid's my_precision: 1
[21]	valid's my_precision: 1
[22]	valid's my_precision: 1
[23]	valid's my_precision: 0.5
[24]	valid's my_precision: 0.666667
[25]	valid's my_precision: 0.666667
[26]	valid's my_precision: 0.666667
[27]	valid's my_precision: 0.6
[28]	valid's my_precision: 0.571429
[29]	valid's my_precision: 0.571429
[30]	valid's my_precision: 0.571429
[31]	valid's my_precision: 0.45

[I 2020-10-29 23:41:32,359] Finished trial#0 with value: 1.0 with parameters: {'lambda_l1': 1.647489685615859e-05, 'lambda_l2': 0.0244375788764034, 'num_leaves': 107, 'feature_fraction': 0.5904889357842075, 'bagging_fraction': 0.591463417137767, 'bagging_freq': 1, 'min_child_samples': 92}. Best is trial#0 with value: 1.0.


[1]	valid's my_precision: 0
Training until validation scores don't improve for 200 rounds
[2]	valid's my_precision: 0
[3]	valid's my_precision: 0
[4]	valid's my_precision: 0
[5]	valid's my_precision: 1
[6]	valid's my_precision: 1
[7]	valid's my_precision: 0.923077
[8]	valid's my_precision: 0.869565
[9]	valid's my_precision: 0.870968
[10]	valid's my_precision: 0.818182
[11]	valid's my_precision: 0.815789
[12]	valid's my_precision: 0.833333
[13]	valid's my_precision: 0.836735
[14]	valid's my_precision: 0.843137
[15]	valid's my_precision: 0.818182
[16]	valid's my_precision: 0.833333
[17]	valid's my_precision: 0.83871
[18]	valid's my_precision: 0.852941
[19]	valid's my_precision: 0.861111
[20]	valid's my_precision: 0.866667
[21]	valid's my_precision: 0.857143
[22]	valid's my_precision: 0.860759
[23]	valid's my_precision: 0.860759
[24]	valid's my_precision: 0.855263
[25]	valid's my_precision: 0.85
[26]	valid's my_precision: 0.845238
[27]	valid's my_precision: 0.845238
[28]	valid's my_precis

[I 2020-10-29 23:41:35,747] Finished trial#1 with value: 1.0 with parameters: {'lambda_l1': 0.24611449704383084, 'lambda_l2': 2.181163327487998e-06, 'num_leaves': 195, 'feature_fraction': 0.6842787096431889, 'bagging_fraction': 0.8211267500330383, 'bagging_freq': 3, 'min_child_samples': 6}. Best is trial#0 with value: 1.0.


[1]	valid's my_precision: 0
Training until validation scores don't improve for 200 rounds
[2]	valid's my_precision: 0
[3]	valid's my_precision: 0
[4]	valid's my_precision: 0
[5]	valid's my_precision: 0
[6]	valid's my_precision: 0
[7]	valid's my_precision: 0
[8]	valid's my_precision: 0
[9]	valid's my_precision: 0
[10]	valid's my_precision: 0
[11]	valid's my_precision: 0
[12]	valid's my_precision: 0
[13]	valid's my_precision: 0
[14]	valid's my_precision: 0
[15]	valid's my_precision: 1
[16]	valid's my_precision: 0.666667
[17]	valid's my_precision: 0.666667
[18]	valid's my_precision: 0.5
[19]	valid's my_precision: 0.5
[20]	valid's my_precision: 0.6
[21]	valid's my_precision: 0.333333
[22]	valid's my_precision: 0.375
[23]	valid's my_precision: 0.454545
[24]	valid's my_precision: 0.416667
[25]	valid's my_precision: 0.416667
[26]	valid's my_precision: 0.533333
[27]	valid's my_precision: 0.5625
[28]	valid's my_precision: 0.555556
[29]	valid's my_precision: 0.555556
[30]	valid's my_precision: 0

...

[I 2020-10-29 23:44:04,854] Finished trial#49 with value: 0.8333333333333334 with parameters: {'lambda_l1': 9.456337165726047e-06, 'lambda_l2': 0.000123260484114562, 'num_leaves': 100, 'feature_fraction': 0.8847243541490805, 'bagging_fraction': 0.5260814201564636, 'bagging_freq': 2, 'min_child_samples': 95}. Best is trial#0 with value: 1.0.


[1]	valid's my_precision: 0
Training until validation scores don't improve for 200 rounds
[2]	valid's my_precision: 0
[3]	valid's my_precision: 0
[4]	valid's my_precision: 0
[5]	valid's my_precision: 0
[6]	valid's my_precision: 0
[7]	valid's my_precision: 0
[8]	valid's my_precision: 0
[9]	valid's my_precision: 0
[10]	valid's my_precision: 0
[11]	valid's my_precision: 0
[12]	valid's my_precision: 0
[13]	valid's my_precision: 1
[14]	valid's my_precision: 0.666667
[15]	valid's my_precision: 0.6
[16]	valid's my_precision: 0.5
[17]	valid's my_precision: 0.625
[18]	valid's my_precision: 0.571429
[19]	valid's my_precision: 0.625
[20]	valid's my_precision: 0.727273
[21]	valid's my_precision: 0.75
[22]	valid's my_precision: 0.642857
[23]	valid's my_precision: 0.666667
[24]	valid's my_precision: 0.652174
[25]	valid's my_precision: 0.681818
[26]	valid's my_precision: 0.68
[27]	valid's my_precision: 0.730769
[28]	valid's my_precision: 0.666667
[29]	valid's my_precision: 0.666667
[30]	valid's my_pr

[I 2020-10-29 23:44:07,667] Finished trial#50 with value: 1.0 with parameters: {'lambda_l1': 1.705418205940138e-06, 'lambda_l2': 0.0010944553825879976, 'num_leaves': 61, 'feature_fraction': 0.6721909255661631, 'bagging_fraction': 0.9523140513393795, 'bagging_freq': 6, 'min_child_samples': 83}. Best is trial#0 with value: 1.0.


[1]	valid's my_precision: 0
Training until validation scores don't improve for 200 rounds
[2]	valid's my_precision: 0
[3]	valid's my_precision: 0
[4]	valid's my_precision: 0
[5]	valid's my_precision: 0
[6]	valid's my_precision: 0
[7]	valid's my_precision: 0
[8]	valid's my_precision: 0
[9]	valid's my_precision: 1
[10]	valid's my_precision: 1
[11]	valid's my_precision: 0.9
[12]	valid's my_precision: 0.909091
[13]	valid's my_precision: 0.785714
[14]	valid's my_precision: 0.714286
[15]	valid's my_precision: 0.733333
[16]	valid's my_precision: 0.75
[17]	valid's my_precision: 0.777778
[18]	valid's my_precision: 0.756757
[19]	valid's my_precision: 0.7
[20]	valid's my_precision: 0.73913
[21]	valid's my_precision: 0.755556


KeyboardInterrupt: 

In [52]:
study.best_trial
dtrain = lgb.Dataset(x_train_val, label=y_train_val)

params = {
        "objective": "multiclass",
        "num_class": 3,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 1234,
    }
params.update(dict(trial.params.items()))

lgb_best_model = lgb.LGBMClassifier(**params)
lgb_best_model.fit(x_train, y_train)
y_pred = lgb_best_model.predict(x_test)
print(confusion_matrix(y_pred, y_test))

ValueError: No trials are completed yet.

## Random Forest

In [37]:
import sklearn.model_selection
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
    
    rf_optuna_model = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, 
                                                              max_depth=max_depth)
    
    score_funcs = {
        'builtin_accuracy': 'accuracy',
        'custom_accuracy': make_scorer(accuracy_score)
    }
    
    return sklearn.model_selection.cross_val_score(
        rf_optuna_model, x_train, y_train, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=100)

trial = study.best_trial

[I 2020-10-29 22:49:56,030] Finished trial#0 with value: 0.6853857468384169 with parameters: {'n_estimators': 12, 'max_depth': 1.1716003972813473}. Best is trial#0 with value: 0.6853857468384169.
[I 2020-10-29 22:49:57,201] Finished trial#1 with value: 0.6933004652059541 with parameters: {'n_estimators': 5, 'max_depth': 4.07186313553001}. Best is trial#1 with value: 0.6933004652059541.
[I 2020-10-29 22:49:57,563] Finished trial#2 with value: 0.7479944730261257 with parameters: {'n_estimators': 9, 'max_depth': 16.561828677192786}. Best is trial#2 with value: 0.7479944730261257.
[I 2020-10-29 22:49:58,752] Finished trial#3 with value: 0.714178862108407 with parameters: {'n_estimators': 15, 'max_depth': 6.4568999055248675}. Best is trial#2 with value: 0.7479944730261257.
[I 2020-10-29 22:49:59,911] Finished trial#4 with value: 0.6853857468384169 with parameters: {'n_estimators': 17, 'max_depth': 1.2346117436321704}. Best is trial#2 with value: 0.7479944730261257.
[I 2020-10-29 22:50:00,26

[I 2020-10-29 22:50:28,677] Finished trial#82 with value: 0.7667006366315593 with parameters: {'n_estimators': 16, 'max_depth': 25.580639157557737}. Best is trial#73 with value: 0.7789598887619666.
[I 2020-10-29 22:50:29,061] Finished trial#83 with value: 0.7696254765359926 with parameters: {'n_estimators': 17, 'max_depth': 22.85250911992644}. Best is trial#73 with value: 0.7789598887619666.
[I 2020-10-29 22:50:29,444] Finished trial#84 with value: 0.7537524942473927 with parameters: {'n_estimators': 18, 'max_depth': 14.754552757409565}. Best is trial#73 with value: 0.7789598887619666.
[I 2020-10-29 22:50:29,802] Finished trial#85 with value: 0.7552032805579763 with parameters: {'n_estimators': 9, 'max_depth': 17.80694803750214}. Best is trial#73 with value: 0.7789598887619666.
[I 2020-10-29 22:50:30,207] Finished trial#86 with value: 0.7537447295304598 with parameters: {'n_estimators': 19, 'max_depth': 29.765496528606818}. Best is trial#73 with value: 0.7789598887619666.
[I 2020-10-29

In [38]:
rf_best_model = sklearn.ensemble.RandomForestClassifier(**study.best_params)
rf_best_model.fit(x_train, y_train)
y_pred = rf_best_model.predict(x_test)
confusion_matrix(y_pred, y_test)